In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input, layers
from layers import SpectralNormalization

def get_generator():
    z = Input(shape=(10,), name='noisy')
    x = z
    x = SpectralNormalization(layers.Dense(20))(x)
    #x = layers.Dense(20)(x)
    outputs = x
    return Model(inputs=[z], outputs=outputs)

In [2]:
mirrored_strategy = tf.distribute.MirroredStrategy()

In [3]:
with mirrored_strategy.scope():
    G = get_generator()
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, beta_1=0.)


sn build False
tf.Tensor(
[[-0.65450794  0.8323131   0.08757466 -2.0011845  -0.03293195  0.18466248
  -0.81771445  1.0800742  -1.3708953   0.47246563  0.78543454  0.8174892
  -0.5517115  -0.77204365  0.1748218   0.2880929   0.74631315 -0.0842401
   2.581838    0.6878972 ]], shape=(1, 20), dtype=float32)
tf.Tensor(
[[-0.65450794  0.8323131   0.08757466 -2.0011845  -0.03293195  0.18466248
  -0.81771445  1.0800742  -1.3708953   0.47246563  0.78543454  0.8174892
  -0.5517115  -0.77204365  0.1748218   0.2880929   0.74631315 -0.0842401
   2.581838    0.6878972 ]], shape=(1, 20), dtype=float32)
build done
sn call Tensor("noisy:0", shape=(None, 10), dtype=float32) False
sn update_uv
W <class 'tensorflow.python.distribute.values.MirroredVariable'> MirroredVariable:{
  0 /job:localhost/replica:0/task:0/device:GPU:0: <tf.Variable 'spectral_normalization/kernel:0' shape=(10, 20) dtype=float32>,
  1 /job:localhost/replica:0/task:0/device:GPU:1: <tf.Variable 'spectral_normalization/kernel/replica_1:

In [4]:
@tf.function
def train_step(G, optimizer):
        # Update G.
        def step_fn():
            train_x = tf.random.normal([2, 10])
            train_y = tf.concat([train_x, train_x[:, ::-1]], axis=1)
            with tf.GradientTape() as gen_tape:
                output = G(train_x)
                gen_loss = tf.losses.mean_squared_error(output, train_y)

            grads_G = gen_tape.gradient(gen_loss, G.trainable_variables)
            optimizer.apply_gradients(zip(grads_G, G.trainable_variables))
        per_example_losses = mirrored_strategy.experimental_run_v2(step_fn)

In [5]:
with mirrored_strategy.scope():
    for i in range(3000):
        train_step(G, optimizer)

sn call Tensor("random_normal:0", shape=(2, 10), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0) False
sn call Tensor("replica_1/random_normal:0", shape=(2, 10), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:1) False
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
sn call Tensor("random_normal:0", shape=(2, 10), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:0) False
sn call Tensor("replica_1/random_normal:0", shape=(2, 10), dtype=float32, device=/job:localhost/replica:0/task:0/device:GPU:1) False
INFO:tensorflow:batch_all_reduce: 2 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10


In [6]:
train_x = tf.random.normal([1, 10])
train_y = tf.concat([train_x, train_x[:, ::-1]], axis=1)
output = G(train_x)

sn call tf.Tensor(
[[-1.96855    -0.7202482  -0.85717434  1.4834418   0.82927316 -0.43805447
   1.2045871  -0.5977998   0.49789014  0.05792151]], shape=(1, 10), dtype=float32) True


In [7]:
output - train_y

<tf.Tensor: id=875, shape=(1, 20), dtype=float32, numpy=
array([[ 1.0457635e-02,  2.4338365e-03,  2.7161837e-04, -6.5629482e-03,
        -7.3384643e-03,  4.7552586e-04, -8.3702683e-02,  4.0884018e-03,
        -1.2866795e-02, -1.1995435e-05, -2.4969876e-04, -1.6444862e-02,
         2.0337105e-04, -4.5251369e-02,  4.6417117e-04, -2.0331144e-04,
        -8.5842609e-02,  1.7935991e-02,  5.7500601e-04,  9.0997934e-02]],
      dtype=float32)>

In [8]:
G.layers[-1]u

AttributeError: 'Model' object has no attribute 'u'